In [1]:
from config.configuration import engine
from tools import idealista_functions as i_func
import pandas as pd
from tools import functions as func
from tools import google_functions as g_func
from datetime import datetime

Conectado a mysql / base de datos: tu_pisito


In [2]:
token = i_func.get_idealista_token()

In [3]:
neighborhoods = [
    "Puerta Bonita",
    "Casco Histórico de Vallecas",
    "Acacias",
    "Pacífico",
    "Castilla",
    "Guindalera",
    "Valdeacederas", 
    "Canillas",
    "Media Legua",
    "Los Ángeles"
]

lista = []

for n in neighborhoods: 
    neighborhood_df = pd.read_sql_query(
    f"""
    SELECT neighborhood_name, latitude, longitude
    FROM neighborhoods
    WHERE neighborhood_name = "{n}";
    """, engine)
    
    neighborhood = neighborhood_df["neighborhood_name"][0]
    lat_lon = str(neighborhood_df["latitude"][0]) + "," + str(neighborhood_df["longitude"][0])
    
    dic = {
        "neighborhood": neighborhood,
        "lat,lon": lat_lon
        }
    
    lista.append(dic)

df = pd.DataFrame(lista)

df

,neighborhood,"lat,lon"
0,Puerta Bonita,"40.3865,-3.73456"
1,Casco Histórico de Vallecas,"40.3773,-3.62171"
2,Acacias,"40.4041,-3.70596"
3,Pacífico,"40.4014,-3.67488"
4,Castilla,"40.4764,-3.68004"
5,Guindalera,"40.4364,-3.66802"
6,Valdeacederas,"40.4679,-3.70342"
7,Canillas,"40.4619,-3.64335"
8,Media Legua,"40.412,-3.65753"
9,Los Ángeles,"40.3562,-3.69964"


In [4]:
result_1 = i_func.idealista_request(token, df["lat,lon"][1], 2000)

In [5]:
df_1_mysql, df_1_pred = i_func.creating_dataframe(result_1)
df_1_mysql.to_csv("./data/dfs_retrain/df_1_mysql.csv", index = False)
df_1_pred.to_csv("./data/dfs_retrain/df_1_pred.csv", index = False)

In [7]:
result_9 = i_func.idealista_request(token, df["lat,lon"][9], 2000)

In [8]:
df_9_mysql, df_9_pred = i_func.creating_dataframe(result_9)
df_9_mysql.to_csv("./data/dfs_retrain/df_9_mysql.csv", index = False)
df_9_pred.to_csv("./data/dfs_retrain/df_9_pred.csv", index = False)

#### Some other tests:

In [2]:
lst = [] 

lst.append({"option": "bar",
        "confirmation": True})
lst.append({"option": "restaurant",
        "confirmation": False})
lst.append({"option": "cafe",
        "confirmation": True})
lst.append({"option": "subway_station",
        "confirmation": False})

In [6]:
df = pd.DataFrame(lst)

In [10]:
df.loc[-1] = ["bus_station", False]
df.index += 1

In [11]:
df

,option,confirmation
1,bar,True
2,restaurant,False
3,cafe,True
4,subway_station,False
0,bus_station,False


In [35]:
df = df[df["confirmation"]]
lst = []
latitude = 40.466884128057536
longitude = -3.689750746220336

for i, row in df.iterrows():
    data = g_func.get_places(latitude, longitude, type_ = row["option"])
    lst = g_func.google_places(data, row["option"], lst)

test = pd.DataFrame(lst)

In [88]:
type(test.loc[1])

pandas.core.series.Series

In [18]:
def help_me_google(place_type, latitude, longitude):
    if df["bar"][0]:
        bars = g_func.get_bars(latitude, longitude)
    if df["restaurant"][0]:
        restaurants = g_func.get_restaurants(latitude, longitude)
    if df["cafe"][0]:
        cafes = g_func.get_cafes(latitude, longitude)
    if df["subway_station"][0]:
        subway_stations = g_func.get_subway_stations(latitude, longitude)
    if df["train_station"][0]:
        train_stations = g_func.get_train_stations(latitude, longitude)
    if df["gym"][0]:
        gyms = g_func.get_gyms(latitude, longitude)
    if df["parking"][0]:
        parkings = g_func.get_parkings(latitude, longitude)
    if df["primary_school"][0]:
        primary_schools = g_func.get_primary_schools(latitude, longitude)
    if df["secondary_school"][0]:
        secondary_schools = g_func.get_secondary_schools(latitude, longitude)
    if df["shopping_mall"][0]:
        shopping_malls = g_func.get_shopping_malls(latitude, longitude)

In [67]:
now = datetime.now()
current_time = now.strftime("%d-%m-%y-%Hh%Mm%Ss")
print(current_time)

29-07-21-11h38m56s


In [68]:
test.to_csv(f"./data/{current_time}.csv", index = False)

#### Transforming df:

In [2]:
apartments_df = pd.read_csv("./data/real_data/test.csv")

In [3]:
predicted_df = func.predict_prices(apartments_df)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 49 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.4
H2O_cluster_version_age:,"21 days, 3 hours and 55 minutes"
H2O_cluster_name:,H2O_from_python_ccastrob_2wx9av
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.960 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [5]:
predicted_df.head()

,name,sq_mt_built,n_rooms,n_bathrooms,latitude,longitude,buy_price,is_new_development,is_renewal_needed,has_lift,is_exterior,has_parking,floor,house_type_id,value_m2,neighborhood,district,url,prediction,buy_prediction
0,Piso en Calle CARACAS,65,2,1,40.430447,-3.693837,338000,0,0,1,0,0,0,0,6564.27,Almagro,Chamberí,https://www.idealista.com/inmueble/94904750/,5.555035,358950
1,Piso,200,3,4,40.425832,-3.681347,2600000,0,0,0,1,0,4,0,8392.43,Recoletos,Salamanca,https://www.idealista.com/inmueble/94729701/,6.360867,2295446
2,"Piso en Calle de Lope de Rueda, 10",75,3,1,40.423056,-3.677494,439900,0,0,1,0,0,6,0,5887.64,Goya,Salamanca,https://www.idealista.com/inmueble/94853917/,5.650212,446901
3,"Piso en Calle PIZARRO, 8",139,3,2,40.422829,-3.705933,750000,0,0,0,1,0,1,0,5196.25,Malasaña-Universidad,Centro,https://www.idealista.com/inmueble/93591362/,5.781196,604221
4,Piso,278,6,3,40.437277,-3.690053,1395000,0,0,1,1,1,2,0,6564.27,Almagro,Chamberí,https://www.idealista.com/inmueble/93380726/,6.186810,1537480


In [13]:
apartments_df.loc[1].astype(str)["buy_price"]

str